In [57]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import sys
import gym
from importlib import reload
from configparser import ConfigParser
from datetime import datetime
import tensorflow as tf
import time


# get the relative path
pref                = os.getcwd()

os.environ["RL_PATH"]   = pref

#os.environ["RL_PATH"] = "/Users/ankitgupta/Documents/git/anks/Books/ReinforcementLearning/DeepQLearning"
#pref = os.environ["RL_PATH"]

if f'{pref}/RLLibrary' not in sys.path:
    sys.path.append(f'{pref}/RLLibrary')


In [2]:


# importing custom libraries
from ActionSelection import ActionExploration
from ConfigReader import Config
#from RLAgents import QLearningAgent, FittedQAgent, DQN, DoubleDQN
import PolicyGradientAgents
import utils as RLUtils
from Agents.PolicyGradient import Reinforce, ActorCritic
from Agents.PolicyGradient import discountRewards
import loggingConfig as loggingConfig


logger = loggingConfig.logging
logger.getLogger("PG_Train")


<Logger PG_Train (INFO)>

In [3]:
reload(Reinforce)

<module 'Agents.PolicyGradient.Reinforce' from '/Users/ankitgupta/Documents/git/anks/MachineLearning/ReinforcementLearning/RLLibrary/Agents/PolicyGradient/Reinforce.py'>

In [4]:
configFile  = os.path.join(pref, "Configs.ini" )
savePath    = os.path.join(os.environ["RL_PATH"], "models" )
_time       = datetime.now().strftime("%Y%m%d%H%M")

env = gym.make("CartPole-v1")
state = env.reset()


In [83]:
modelParams = {"Name": "Policy", "NetworkShape": [1024, 512], "learning_rate": 0.01, \
                "optimizer": "ADAM", "loss": "categorical_crossentropy", }
valueParams = {"Name_value": "Value", "NetworkShape_value": [24, 12], "learning_rate_value": 0.0001, \
                "optimizer_value": "ADAM", "loss_value": "mse", }






In [139]:
reload(ActorCritic)

<module 'Agents.PolicyGradient.ActorCritic' from '/Users/ankitgupta/Documents/git/anks/MachineLearning/ReinforcementLearning/RLLibrary/Agents/PolicyGradient/ActorCritic.py'>

In [140]:
ReinforceAgent   = ActorCritic.ActorCritic(env = env, configFile = configFile, **modelParams)


In [141]:
ReinforceAgent.SharedNetwork.optimizer

In [142]:
ReinforceAgent.SharedNetwork.summary()

Model: "SharedDesign"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
dense_32 (Dense)                (None, 1024)         5120        input_9[0][0]                    
__________________________________________________________________________________________________
dense_33 (Dense)                (None, 512)          524800      dense_32[0][0]                   
__________________________________________________________________________________________________
dense_34 (Dense)                (None, 2)            1026        dense_33[0][0]                   
_______________________________________________________________________________________

In [143]:
# lets run for 1 episode

In [69]:
import tensorflow as tf

(array([ 0.04276836, -0.01055716,  0.03850685, -0.00280508]),
 0,
 1.0,
 array([ 0.04255722, -0.20620959,  0.03845075,  0.30177423]),
 False,
 array([0.49938238, 0.5006177 ], dtype=float32),
 -0.0016929482)

In [119]:
Agent.memory

[(array([ 0.02299997, -0.1845979 , -0.00340317,  0.26377976]),
  0,
  1.0,
  array([ 0.01930802, -0.37967111,  0.00187243,  0.55538736]),
  False,
  array([0.5164557 , 0.48354435], dtype=float32),
  8.142222)]

In [120]:
with tf.GradientTape() as tape:

    
    for sample in Agent.memory:
        
        # get log prob
        state   = tf.Variable([sample[0]], trainable = True,   dtype=tf.float32)
        nextstate   = tf.Variable([sample[3]], trainable = True,   dtype=tf.float32)
        action, reward, dead  = sample[1], sample[2], sample[4]
        

        #nextState = tf.convert_to_tensor([nextState], dtype=tf.float32)

        # run shared network to get action prob distro and value associated with that state
        actionProbDistro, _currStateValue = Agent.SharedNetwork(state, training = True)
        _actionProbDistro, _nextStateValue = Agent.SharedNetwork(nextState, training = True)
        
        #print(actionProbDistro, _currStateValue, _actionProbDistro, _nextStateValue)

        # compute TD error
        # delta = r + gamma*V_(t+1) - V_t
        delta = reward + Agent.discountfactor* tf.squeeze(_nextStateValue) * (1 - int(dead)) - tf.squeeze(_currStateValue)
        print(tf.squeeze(_nextStateValue), tf.squeeze(_currStateValue), delta)

        # compute critic loss
        loss_sample_critic = delta**2

        # compute actor loss
        print(actionProbDistro)
        
        actionProb      = actionProbDistro.numpy()[0, action]
        print("action rob for action: ", action, "Prob: ", actionProb)
        loss_sample_actor =  tf.math.log(actionProb) * delta
        print("losses", loss_sample_critic.numpy(), -loss_sample_actor.numpy())
        
                
        print(loss_actor, loss_critic)


    networkLoss = sum(loss_critic) + sum(loss_actor)

    logger.info(f"{Agent.Name} - Updating Policy ")

    # performing Backpropagation to update the network
    
    grads = tape.gradient(networkLoss, Agent.SharedNetwork.trainable_variables)
    print(grads)
    Agent.SharedNetwork.optimizer.apply_gradients(zip(grads, Agent.SharedNetwork.trainable_variables))

    # reset memory
    #Agent.memory = []

tf.Tensor(7.996466, shape=(), dtype=float32) tf.Tensor(8.142222, shape=(), dtype=float32) tf.Tensor(0.0545969, shape=(), dtype=float32)
tf.Tensor([[0.5164557  0.48354435]], shape=(1, 2), dtype=float32)
action rob for action:  0 Prob:  0.5164557
losses 0.0029808215 0.036075763
[<tf.Tensor: id=12728, shape=(), dtype=float32, numpy=0.6921267>] []


ValueError: No gradients provided for any variable: ['dense_24/kernel:0', 'dense_24/bias:0', 'dense_25/kernel:0', 'dense_25/bias:0', 'dense_26/kernel:0', 'dense_26/bias:0', 'dense_27/kernel:0', 'dense_27/bias:0'].

In [132]:
grads

[None, None, None, None, None, None, None, None]

In [72]:
[state

[<tf.Tensor: id=2740, shape=(4,), dtype=float32, numpy=array([ 0.04276836, -0.01055716,  0.03850685, -0.00280508], dtype=float32)>]

In [53]:
_currentState

array([-0.03763838, -0.03630947, -0.01659229, -0.03680008])

In [54]:
_currentState = _currentState.reshape([1, _currentState.shape[0]])
_currentState

array([[-0.03763838, -0.03630947, -0.01659229, -0.03680008]])

In [55]:
act, val = Agent.SharedNetwork(_currentState)

In [57]:
act

<tf.Tensor: id=1686, shape=(1, 2), dtype=float32, numpy=array([[0.49874383, 0.5012562 ]], dtype=float32)>

In [56]:
np.squeeze(act)

array([0.49874383, 0.5012562 ], dtype=float32)

In [34]:
act /= tf.reduce_sum(act)

In [51]:
val.numpy()[0][0]

-0.0007066454

In [30]:
tf.reduce_sum(act)


<tf.Tensor: id=1131, shape=(), dtype=float32, numpy=1.0>

In [40]:
action             = np.random.choice(Agent.env.action_space.n, p = act.numpy()[0])
action

0

In [39]:
# for all experience in batchsize
curStates       = np.vstack(list(list(zip(*Agent.memory)))[0])
actions         = np.vstack(list(list(zip(*Agent.memory)))[1])
nextStates      = np.vstack(list(list(zip(*Agent.memory)))[3])
rewards         = np.vstack(list(list(zip(*Agent.memory)))[2])
done            = np.vstack(list(list(zip(*Agent.memory)))[4])

actionProb      = np.vstack(list(list(zip(*Agent.memory)))[5])


IndexError: list index out of range

In [28]:
# compute the discounted rewards for the entire episode and normalize it
discounted_rewards = discountRewards(rewards,  discountfactor=Agent.discountfactor)


In [30]:
discounted_rewards = (discounted_rewards - np.mean(discounted_rewards))/ (np.std(discounted_rewards) + 1e-7)            # to avoid division by 0

In [33]:
discounted_rewards[4][0]

0.9905667496168143

In [61]:
losses = []

with tf.GradientTape() as tape:
    for index, sample in enumerate(Agent.memory):

        # get log prob
        state = tf.Variable([sample[0]], trainable=True, dtype=tf.float64)
        prob = Agent.PolicyNetwork.model(state, training = True)
        action = sample[1]
        actionProb = prob[0, action]
        logProb = tf.math.log(actionProb)

        #actionProb = sample[5]
        #logProb = np.log(actionProb)
        sampleloss = logProb * discounted_rewards[index][0]

        losses.append(-sampleloss)      # this is negative, since we are interested in gradient ascent

    networkLoss = sum(losses)
    
    grads = tape.gradient(networkLoss, Agent.PolicyNetwork.model.trainable_variables)
    Agent.PolicyNetwork.optimizer.apply_gradients(zip(grads, Agent.PolicyNetwork.model.trainable_variables))



In [51]:
sample

(array([-0.12274747, -1.90964119,  0.19205285,  2.89212446]),
 1,
 1.0,
 array([-0.16094029, -1.7161645 ,  0.24989534,  2.66336226]),
 True,
 array([0.7354947 , 0.26450533], dtype=float32))

In [50]:
prob

<tf.Tensor: id=5343, shape=(1, 2), dtype=float32, numpy=array([[0.7354947, 0.2645053]], dtype=float32)>

In [52]:
action = sample[1]
actionProb = prob[0, action]
actionProb

<tf.Tensor: id=5352, shape=(), dtype=float32, numpy=0.2645053>

In [48]:
logProb

<tf.Tensor: id=5344, shape=(1, 2), dtype=float32, numpy=array([[-0.307212, -1.329894]], dtype=float32)>

AttributeError: 'numpy.dtype' object has no attribute 'is_floating'

In [ ]:

# compute the discounted rewards for the entire episode and normalize it
discounted_rewards = discountRewards(rewards,  discountfactor=self.discountfactor)
if self.normalizeRewards:
    discounted_rewards = (discounted_rewards - np.mean(discounted_rewards))/ (np.std(discounted_rewards) + 1e-7)            # to avoid division by 0


#TODO -----> add eager execution
losses = []

with tf.GradientTape() as tape:
    for curStates, actions, nextStates, rewards, done, actionProb in self.memory:

        # get log prob
        logProb = np.log(actionProb)
        sampleloss = logProb * discounted_rewards

        losses.append(-sampleloss)      # this is negative, since we are interested in gradient ascent

    networkLoss = np.sum(losses)

grads = tape.gradient(networkLoss, self.PolicyNetwork.model.trainable_variables)
self.PolicyNetwork.optimizer.apply_gradients(zip(grads, model.trainable_variables))


In [48]:

# for all experience in batchsize
curStates       = np.vstack(list(list(zip(*Agent.memory)))[0])
actions         = np.vstack(list(list(zip(*Agent.memory)))[1])
nextStates      = np.vstack(list(list(zip(*Agent.memory)))[3])
rewards         = np.vstack(list(list(zip(*Agent.memory)))[2])
done            = np.vstack(list(list(zip(*Agent.memory)))[4])

actionProb      = np.vstack(list(list(zip(*Agent.memory)))[5])


# Get X
try:
    Agent.env.observation_space.n
    X_train = getOneHotrepresentation(curStates, num_classes=Agent.inputShape)
except:
    X_train = curStates    


In [49]:
X_train[:4], actionProb[:4]

(array([[-0.03113103, -0.02077829,  0.00398832,  0.00614775],
        [-0.03154659,  0.17428624,  0.00411127, -0.28527415],
        [-0.02806087, -0.0208941 , -0.00159421,  0.0087026 ],
        [-0.02847875,  0.17425067, -0.00142016, -0.28448289]]),
 array([[0.5012765 , 0.49872348],
        [0.5122483 , 0.48775175],
        [0.50089973, 0.49910033],
        [0.51175475, 0.4882453 ]], dtype=float32))

In [50]:
discounted_rewards = discountRewards(rewards,  discountfactor=Agent.discountfactor)
discounted_rewards[:4]

[array([14.85422289]),
 array([13.99416454]),
 array([13.12541872]),
 array([12.2478977])]

In [51]:
if Agent.baseline.upper() == "VALUE":
    value = Agent.ValueNetwork.model.predict(X_train)
    G = discounted_rewards - value


elif Agent.baseline.upper() == "NORMALIZE":
    G = discounted_rewards
    G = (G - np.mean(G))/ (np.std(G) + 1e-7)            # to avoid division by 0

G[:4]    

array([[1.58898873],
       [1.38696682],
       [1.1829043 ],
       [0.97678053]])

In [52]:
gradient = np.subtract(RLUtils.getOneHotrepresentation(actions,Agent.env.action_space.n ), actionProb)
gradient[:4]

array([[-0.5012765 ,  0.5012765 ],
       [ 0.48775172, -0.48775175],
       [-0.50089973,  0.5008997 ],
       [ 0.48824525, -0.4882453 ]], dtype=float32)

In [53]:
gradient *= G 
gradient[:4]

array([[-0.7965227 ,  0.7965227 ],
       [ 0.67649543, -0.6764955 ],
       [-0.5925164 ,  0.59251636],
       [ 0.47690845, -0.4769085 ]], dtype=float32)

In [54]:
gradient *= Agent.policy_learning_rate
gradient[:4]

array([[-7.9652265e-05,  7.9652265e-05],
       [ 6.7649540e-05, -6.7649547e-05],
       [-5.9251641e-05,  5.9251634e-05],
       [ 4.7690843e-05, -4.7690850e-05]], dtype=float32)

In [40]:
import copy

In [55]:
# old model
old = tf.keras.models.clone_model(Agent.PolicyNetwork.model)

In [56]:

# updating actual probabilities (y_train) to take into account the change in policy gradient change
# \theta = \theta + alpha*rewards * gradient
y_train = actionProb + np.vstack(gradient)



# update policy and also learn the value function

# 1. Update policy
history = Agent.PolicyNetwork.model.train_on_batch(X_train, y_train)

Agent.memory = []

In [58]:
old.predict(X_train[:4])

array([[0.49950963, 0.50049037],
       [0.5345103 , 0.46548966],
       [0.49908236, 0.5009177 ],
       [0.53361577, 0.4663842 ]], dtype=float32)

In [59]:
Agent.PolicyNetwork.model.predict(X_train[:4])

array([[0.5013335 , 0.49866652],
       [0.5123426 , 0.4876574 ],
       [0.5009569 , 0.4990431 ],
       [0.51184905, 0.48815095]], dtype=float32)

In [1]:
import tensorflow_probability as tfp

ModuleNotFoundError: No module named 'tensorflow_probability'

In [3]:
actions = [[0. ,1.],
 [0. ,1.],
 [1. ,0.],
 [0. ,1.],
 [0. ,1.]]

action = tf.convert_to_tensor(actions, dtype = tf.float32)
action

<tf.Tensor: id=1, shape=(5, 2), dtype=float32, numpy=
array([[0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.]], dtype=float32)>

In [10]:
actions = [0,0,1,1,1]

prob = [[0.49889222, 0.5011078 ],
 [0.5098192 , 0.4901808 ],
 [0.49936554, 0.5006344 ],
 [0.51149035, 0.48850965],
 [0.524573   ,0.4754269 ]]
probs = tf.convert_to_tensor(prob, dtype = tf.float32)
probs

<tf.Tensor: id=3, shape=(5, 2), dtype=float32, numpy=
array([[0.49889222, 0.5011078 ],
       [0.5098192 , 0.4901808 ],
       [0.49936554, 0.5006344 ],
       [0.51149035, 0.48850965],
       [0.524573  , 0.4754269 ]], dtype=float32)>

In [9]:
np.array(actions)*np.array(prob)

array([[0.        , 0.5011078 ],
       [0.        , 0.4901808 ],
       [0.49936554, 0.        ],
       [0.        , 0.48850965],
       [0.        , 0.4754269 ]])

In [11]:
prob_action = []
for index, _prob in enumerate(probs.numpy()):
    prob_action.append(_prob[actions[index]])
    
prob_action

[0.49889222, 0.5098192, 0.5006344, 0.48850965, 0.4754269]

In [12]:
prob_action = tf.convert_to_tensor(np.array(prob_action).reshape(-1,1), dtype = tf.float32)
prob_action

<tf.Tensor: id=4, shape=(5, 1), dtype=float32, numpy=
array([[0.49889222],
       [0.5098192 ],
       [0.5006344 ],
       [0.48850965],
       [0.4754269 ]], dtype=float32)>

In [13]:
advantages = [[4.8847766],
 [3.933328 ],
 [2.9502857],
 [1.9861633],
 [0.9786838]]

advantages = tf.convert_to_tensor(advantages, dtype = tf.float32)
advantages, advantages.shape

(<tf.Tensor: id=5, shape=(5, 1), dtype=float32, numpy=
 array([[4.8847766],
        [3.933328 ],
        [2.9502857],
        [1.9861633],
        [0.9786838]], dtype=float32)>,
 TensorShape([5, 1]))

In [16]:
tf.math.log(prob_action + 1e-10)

<tf.Tensor: id=24, shape=(5, 1), dtype=float32, numpy=
array([[-0.6953652 ],
       [-0.6736991 ],
       [-0.6918792 ],
       [-0.71639603],
       [-0.74354213]], dtype=float32)>

In [17]:
-tf.stop_gradient(advantages) 

<tf.Tensor: id=26, shape=(5, 1), dtype=float32, numpy=
array([[-4.8847766],
       [-3.933328 ],
       [-2.9502857],
       [-1.9861633],
       [-0.9786838]], dtype=float32)>

In [21]:
print("shaep after", prob_action.shape)
loss_actor = tf.reduce_sum(-tf.stop_gradient(advantages) * tf.math.log(prob_action + 1e-10), axis=0)

loss_actor

shaep after (5, 1)


<tf.Tensor: id=50, shape=(1,), dtype=float32, numpy=array([10.238397], dtype=float32)>

In [20]:
temp = [3.3967037 , 2.6498795 , 2.0412414 , 1.4228795 , 0.72769266]
sum(temp)

10.238396760000002

In [23]:
# entropy loss
loss_entropy = tf.reduce_sum(-1 * prob_action * tf.math.log(prob_action + 1e-10), axis=0)

loss_entropy

<tf.Tensor: id=66, shape=(1,), dtype=float32, numpy=array([1.740222], dtype=float32)>

In [30]:
# total loss
total_loss = tf.reduce_mean(loss_actor + loss_entropy + loss_critic)
total_loss

<tf.Tensor: id=90, shape=(), dtype=float32, numpy=22.56641>

In [25]:
loss_critic = tf.reduce_mean(tf.math.square(advantages))
loss_critic

<tf.Tensor: id=70, shape=(), dtype=float32, numpy=10.587792>

In [28]:
total_loss = loss_actor + loss_entropy + loss_critic
total_loss

<tf.Tensor: id=80, shape=(1,), dtype=float32, numpy=array([22.56641], dtype=float32)>

In [ ]:
##---------tf.Tensor(20.853111, shape=(), dtype=float32)

In [ ]:

        # forward run the netwrok to get probabilities and value functions for all states in memory 
        probs, values = self.localModel(tf.convert_to_tensor(np.vstack(memory.states), dtype = tf.float32))

        advantages = discounted_rewards - values.numpy()
        advantages = tf.convert_to_tensor(advantages, dtype = tf.float32)

        # critic loss
        loss_critic = tf.reduce_mean(tf.math.square(advantages))

        # actor loss
        onehotActions = tf.one_hot(memory.actions, self.action_size, dtype =tf.float32)

        # prob of action taken
        prob_action = []
        for index, _prob in enumerate(probs.numpy()):
            prob_action.append(_prob[memory.actions[index]])

        prob_action = tf.convert_to_tensor(np.array(prob_action).reshape(-1,1), dtype = tf.float32)
        loss_actor = tf.reduce_sum(-tf.stop_gradient(advantages) * tf.math.log(prob_action + 1e-10), axis=0)

        # entropy loss
        loss_entropy = tf.reduce_sum(-1 * prob_action * tf.math.log(prob_action + 1e-10), axis=0)

        # total loss
        total_loss = tf.reduce_mean(loss_actor + beta_entropy*loss_entropy + loss_critic)
        print(total_loss)
        return total_loss



In [34]:
# critic loss
loss_critic = tf.math.square(advantages)
print("loss critic",loss_critic)
# prob of action taken
prob_action = []
for index, _prob in enumerate(probs.numpy()):
    prob_action.append(_prob[actions[index]])

prob_action = tf.convert_to_tensor(np.array(prob_action).reshape(-1,1), dtype = tf.float32)

loss_actor = -tf.stop_gradient(advantages) * tf.math.log(prob_action + 1e-10)
print("action loss",loss_actor)

# entropy loss
loss_entropy = -1 * prob_action * tf.math.log(prob_action + 1e-10)
print("entropy loss",loss_entropy)

# total loss
total_loss = tf.reduce_mean(loss_actor + loss_entropy + loss_critic)
print("mean loss",total_loss)

loss critic tf.Tensor(
[[23.861042 ]
 [15.471068 ]
 [ 8.7041855]
 [ 3.9448445]
 [ 0.957822 ]], shape=(5, 1), dtype=float32)
action loss tf.Tensor(
[[3.3967037 ]
 [2.6498795 ]
 [2.0412414 ]
 [1.4228795 ]
 [0.72769266]], shape=(5, 1), dtype=float32)
entropy loss tf.Tensor(
[[0.3469123 ]
 [0.34346473]
 [0.3463785 ]
 [0.34996638]
 [0.35349995]], shape=(5, 1), dtype=float32)
mean loss tf.Tensor(12.983516, shape=(), dtype=float32)


In [35]:
loss_actor + loss_entropy + loss_critic

<tf.Tensor: id=160, shape=(5, 1), dtype=float32, numpy=
array([[27.604658 ],
       [18.464413 ],
       [11.091805 ],
       [ 5.7176905],
       [ 2.0390146]], dtype=float32)>

In [40]:
from Agents.PolicyGradient.A3C import Agents

In [46]:
reload(Agents)

<module 'Agents.PolicyGradient.A3C.Agents' from '/Users/ankitgupta/Documents/git/anks/MachineLearning/ReinforcementLearning/RLLibrary/Agents/PolicyGradient/A3C/Agents.py'>

In [48]:
save_dir         = f"{pref}/models/Policy/A3C"
optimizer_args = {"learning_rate": 1e-4}
model = Agents.MasterAgent(game = "CartPole-v0", save_dir=save_dir, **optimizer_args)

#model.train()

In [54]:
from queue import Queue
import multiprocessing
reward_queue = Queue()

global_episode_index = multiprocessing.Value("i", 0)

In [55]:
local = Agents.WorkerAgent(state_size = model.state_size, action_size = model.action_size, \
                    global_model = model.globalModel, sharedOptimizer = model.optimizer, \
                    result_queue = reward_queue, global_episode_index = global_episode_index, \
                    workerIndex = 1, gameName = model.gameName , save_dir = model.save_dir)

In [57]:
local.localModel

In [208]:

def run(local):

    # runs the Agent forever (until all games played)
    # trains the agent while sharing the network parameters 
    local.memory = Agents.Memory()
    total_steps = 1

    while local.worker_episode_index.value < 2000:

        # run the agent for each episode

        current_state = local.env.reset()
        local.memory.reset()

        episodic_reward = 0
        episodic_steps= 0
        local.episodicLoss = 0

        done = False

        while not done:
            probs, _ = local.localModel(tf.convert_to_tensor(current_state[None, :], dtype = tf.float32))
            #print("current_state after", tf.convert_to_tensor(current_state[None, :]))
            action = np.random.choice(local.action_size, p = probs.numpy()[0])  # since probs is a tensor. need to convert to array

            # take a step towards the action
            next_state, reward, done, _ = local.env.step(action)

            if done:
                reward = -1

            episodic_reward += reward

            # store the transition in memory
            local.memory.update(current_state, action, reward)


            if total_steps%5 == 0 or done:
                # --- this is where the local model is being trained and global model gets udpated

                # track the variables involved in loss computation using GradientTape (eager execution)
                with tf.GradientTape() as tape:

                    # get the reward value at final step
                    if done:
                        reward_sum = 0
                    else:
                        _, values = local.localModel(tf.convert_to_tensor(next_state[None, :], dtype=tf.float32))
                        reward_sum = values.numpy()[0]

                    # Here we are assuming 1 step return

                    # get discounted returns
                    discounted_rewards = []
                    discount_factor = 0.99

                    for reward in local.memory.rewards[::-1]:

                        reward_sum = reward + discount_factor*reward_sum
                        discounted_rewards.append(reward_sum)

                    discounted_rewards.reverse()
                    discounted_rewards = np.array(discounted_rewards).reshape(-1,1)


                    # forward run the netwrok to get probabilities and value functions for all states in memory 
                    probs, values = local.localModel(tf.convert_to_tensor(np.vstack(local.memory.states), dtype = tf.float32))

                    advantages = discounted_rewards - values
                    advantages = tf.convert_to_tensor(advantages, dtype = tf.float32)


                    # critic loss
                    loss_critic = tf.math.square(advantages)

                    # actor loss
                    
                    actions= [[index, item] for index, item in enumerate(local.memory.actions)]
                    prob_action = tf.convert_to_tensor(tf.gather_nd(probs, actions)[:, None])
                    #prob_action = tf.gather_nd(probs, actions)
                    print("actionprob", prob_action.shape)
                    
                    # prob of action taken

                    loss_actor_new = tf.math.log(prob_action + 1e-10)
                    print("loss_actor", loss_actor_new)

                    loss_actor_new *= - tf.stop_gradient(advantages)
                    print("loss_actor", loss_actor_new)

                #     loss_actor = tf.reduce_sum(-tf.stop_gradient(advantages) * tf.math.log(probs + 1e-10), axis=1)

                    # entropy loss
                    loss_entropy = tf.math.log(prob_action + 1e-10)
                    loss_entropy *= - prob_action

                    #loss_entropy = tf.reduce_sum(-1 * probs * tf.math.log(probs + 1e-10), axis=1)

                    # total loss
                    print("total loss", loss_critic + loss_actor_new)
                    total_loss = tf.reduce_mean(loss_entropy)
                    print("total loss mean", total_loss)
                    #total_loss = tf.reduce_mean(loss_actor_new + beta_entropy*loss_entropy + loss_critic)
                    

                    #     loss_actor = tf.reduce_sum(-tf.stop_gradient(advantages) * tf.math.log(probs + 1e-10), axis=1)

                        # entropy loss
                    #     loss_entropy = tf.math.log(prob_action + 1e-10)
                    #     loss_entropy *= - prob_action

                        #loss_entropy = tf.reduce_sum(-1 * probs * tf.math.log(probs + 1e-10), axis=1)
                    #print([var.name for var in tape.watched_variables()])
                local.episodicLoss += total_loss

                # compute the gradients with respect to local model
                grads = tape.gradient(total_loss, local.localModel.trainable_weights)
                for grad in grads:
                    if grad is None:
                        print("None grad")
                    else:
                        print(grad.shape)
                
                #print(total_loss)
                #print(grads)

                sys.exit(0)


            total_steps += 1
            current_state = next_state

            episodic_steps += 1                 # we are not using it


        logger.info(f'{self.Name} Episode: {self.worker_episode_index.value}  Reward: {episodic_reward}.   Best Score (GLOBAL): {WorkerAgent.best_score}  ')

        with local.worker_episode_index.get_lock():
            # update the global episode numner
            local.result_queue.put(episodic_reward)
            local.worker_episode_index.value += 1


    # once the agent has exhausted all runs
    local.result_queue.put(None)



In [209]:
run(local)

actionprob (5, 1)
loss_actor tf.Tensor(
[[-0.68606836]
 [-0.7007931 ]
 [-0.68661636]
 [-0.68586445]
 [-0.6801466 ]], shape=(5, 1), dtype=float32)
loss_actor tf.Tensor(
[[3.389187 ]
 [2.7817233]
 [2.0702212]
 [1.3867172]
 [0.6913328]], shape=(5, 1), dtype=float32)
total loss tf.Tensor(
[[27.792925]
 [18.537806]
 [11.161086]
 [ 5.474608]
 [ 1.724497]], shape=(5, 1), dtype=float32)
total loss mean tf.Tensor(0.3457437, shape=(), dtype=float32)
(4, 128)
(128,)
(128, 2)
(2,)
None grad
None grad
None grad
None grad


SystemExit: 0

In [132]:
probs, values = local.localModel(tf.convert_to_tensor(np.vstack(local.memory.states), dtype = tf.float32))

In [133]:
probs

<tf.Tensor: id=7751, shape=(5, 2), dtype=float32, numpy=
array([[0.5020227 , 0.49797735],
       [0.50882125, 0.49117878],
       [0.51637334, 0.48362663],
       [0.5243915 , 0.47560853],
       [0.5181277 , 0.4818724 ]], dtype=float32)>

In [176]:
local.memory.actions

[1, 1, 1, 1, 0]

In [177]:
actions= [[index, item] for index, item in enumerate(local.memory.actions)]
actions

[[0, 1], [1, 1], [2, 1], [3, 1], [4, 0]]

In [197]:
tf.gather_nd(probs, actions)

<tf.Tensor: id=9958, shape=(5,), dtype=float32, numpy=
array([0.49797735, 0.49117878, 0.48362663, 0.47560853, 0.5181277 ],
      dtype=float32)>

In [194]:
a = tf.convert_to_tensor(tf.gather_nd(probs, actions)[:, None])

In [195]:
a.shape

TensorShape([5, 1])

In [196]:
a

<tf.Tensor: id=9956, shape=(5, 1), dtype=float32, numpy=
array([[0.49797735],
       [0.49117878],
       [0.48362663],
       [0.47560853],
       [0.5181277 ]], dtype=float32)>

In [7]:
from tensorflow.keras import models, layers


In [8]:
from tensorflow.keras.layers import Conv2D, MaxPool2D

In [37]:
model = models.Sequential()
model.add(layers.Conv2D(2, (1,3), activation='relu', input_shape=(11, 50,3)))
model.add(layers.Conv2D(1, (1,48), activation='relu'))


In [38]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 11, 48, 2)         20        
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 11, 1, 1)          97        
Total params: 117
Trainable params: 117
Non-trainable params: 0
_________________________________________________________________


In [46]:
#help(layers.RNN)

In [47]:
model = models.Sequential()
model.add(layers.LSTM(units = 20, input_shape=(11, 50)))


In [40]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20)                5680      
Total params: 5,680
Trainable params: 5,680
Non-trainable params: 0
_________________________________________________________________


In [60]:
# -----LSTM training
import os

LIB_PATH = os.environ["RL_PATH"]
LOG_PATH = f'{LIB_PATH}/../logs/'
DATA_PATH = "/Users/ankitgupta/Documents/git/anks/MachineLearning/ReinforcementLearning/UseCases/Port_2Stock/data"

_path = os.path.join(DATA_PATH, f"APA.csv")
_thisData = pd.read_csv(_path, index_col = "Date")

In [61]:
_thisData.head()

,Adj Close
Date,
2003-12-31,32.783871
2004-01-02,32.500908
2004-01-05,32.969822
2004-01-06,33.115353
2004-01-07,32.945576


In [54]:
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler(feature_range=(0,1))

In [63]:
training_set = mm.fit_transform(_thisData)

In [73]:
x_train = []
y_train = []
for i in range(60, 3000):
    x_train.append(training_set[i-60:i,0])
    y_train.append(training_set[i,0])
    
  

    

In [88]:
X_train = np.array(x_train)
Y_train = np.array(y_train)
X_train.shape
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train.shape

(2940, 60, 1)

In [100]:
# create model
model = models.Sequential()
model.add(layers.LSTM(units = 20,  input_shape=(X_train.shape[1], 1)))
model.add(layers.Dropout(.2))


#model.add(layers.LSTM(units = 50))
#model.add(layers.Dropout(.2))
"""
model.add(layers.LSTM(units = 50, return_sequences=True))
model.add(layers.Dropout(.2))
"""
model.add(layers.Dense(units = 1))
model.summary()


Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 20)                1760      
_________________________________________________________________
dropout_6 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 21        
Total params: 1,781
Trainable params: 1,781
Non-trainable params: 0
_________________________________________________________________


In [101]:
model.compile(optimizer = "adam", loss = "mean_squared_error")

In [103]:
model.fit(X_train, Y_train, epochs = 1, batch_size=32)

Train on 2940 samples
2940/2940 [==============================] - 15s 5ms/sample - loss: 0.0584


In [104]:
test = X_train[100]
model.predict(test.reshape((1,60,1)))

array([[0.29918742]], dtype=float32)

In [105]:
Y_train[100]

0.2707841246583246

In [108]:
#create new model
new_model = models.Sequential()
new_model.add(layers.LSTM(units = 20,  input_shape=(X_train.shape[1], 1)))
#new_model.add(layers.Dropout(.2))
#set weights of the first layer
new_model.set_weights(model.layers[0].get_weights())


In [109]:
new_model.predict(test.reshape((1,60,1)))

array([[-0.08292898, -0.11602986,  0.0293291 ,  0.06014059,  0.07082775,
        -0.09266429, -0.12714677,  0.13329615,  0.10601458,  0.1527101 ,
        -0.05965486, -0.04044554,  0.17166282,  0.01756974, -0.01239254,
        -0.07096303,  0.14257011, -0.04356703,  0.19897966,  0.04398943]],
      dtype=float32)

In [110]:
#create new model
new_model = models.Sequential()
new_model.add(layers.LSTM(units = 20,  input_shape=(X_train.shape[1], 1)))
new_model.add(layers.Dropout(.2))
#set weights of the first layer
new_model.set_weights(model.layers[0].get_weights())
new_model.predict(test.reshape((1,60,1)))

array([[-0.08292898, -0.11602986,  0.0293291 ,  0.06014059,  0.07082775,
        -0.09266429, -0.12714677,  0.13329615,  0.10601458,  0.1527101 ,
        -0.05965486, -0.04044554,  0.17166282,  0.01756974, -0.01239254,
        -0.07096303,  0.14257011, -0.04356703,  0.19897966,  0.04398943]],
      dtype=float32)